In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import binance
import ta


%matplotlib inline 

# data analysis
import numpy as np
import pandas as pd
import pandas_ta
import matplotlib.pyplot as plt
import plotly.graph_objects as go

# data communication
import configparser
from typing import Optional, Dict, Any
from requests import Session, Request, Response

# utilities
import math
import logging
import apscheduler
from apscheduler.schedulers.blocking import BlockingScheduler
import unittest
import calendar
from datetime import datetime, timedelta

In [2]:
class BinanceClient:
    """Provides an interface for the trading bot to communicate with the Binance exchange via its' API."""
    
    def __init__(self, endpoint):
        self.maker_fees_USD_futures = 0.0002
        self.taker_fees_USD_futures = 0.0004
        self.session = None
        self.config = None
        self.api_key = None
        self.api_secret = None
        self.endpoint = endpoint
        
        
    def open_session(self):
        """Opens a session with the Binance server."""
        
        self.session = Session()
        
        
    def read_config(self):
        """Initializes the Binance client."""
        
        self.config = configparser.ConfigParser()
        self.config.read('config.ini')
        
        self.api_key = self.config['BINANCE API']['api_key']
        self.api_secret = self.config['BINANCE API']['api_secret']
        
        
    def process_response(self, response: Response) -> Any:
        """Processes the response the server sends to the clients request."""
        
        try:
            data = response.json()
        except ValueError:
            response.raise_for_status()
            raise
        else:
            return data
    
    
    def sign_request(self, request: Request) -> None:
        """Signs confidential requests that need the users API key and API secret."""
        
        ts = int(time.time() * 1000)
        request.params['timestamp'] = str(ts)
        signature_payload = urlencode(request.params).encode("utf-8")
        signature = hmac.new(BINANCE_API_SECRET.encode("utf-8"), signature_payload, hashlib.sha256).hexdigest()
        request.headers['X-MBX-APIKEY'] = self.api_key
        request.params['signature'] = signature
        
        
    def request(self, method: str, path: str,**kwargs,) -> Any:
        """Executes a non-confidential request to the Binance server."""
        
        request = Request(method, self.endpoint + path, **kwargs)
        response = self.session.send(request.prepare())
        return self.process_response(response)
    
    
    def signed_request(self, method: str, path: str, params, **kwargs) -> Any:
        """Executes a confidential request to the Binance server"""
        
        request = Request(method, self.endpoint + path, params=params, **kwargs)
        self.sign_request(request)
        response = session.send(request.prepare())
        return self.process_response(response)
        
        
    def get(self, path: str, params: Optional[Dict[str, Any]] = None) -> Any:
        """Executes an HTTPS GET request."""
        
        return self.request('GET', path, params=params)
    
    
    def post(self, path: str, params: Optional[Dict[str, Any]] = None) -> Any:
        """Executes an HTTPS POST request."""
        
        return self.signed_request('POST', path, params=params)
    
    
    def delete(self, path: str, params: Optional[Dict[str, Any]] = None) -> Any:
        """Executes an HTTPS DELETE request."""
        
        return self.signed_request('DELETE', path, json=params)
    
    
    def place_order(self, market: str, side: str, size: float, reduce_only: bool = False,
                    order_type: str = 'MARKET') -> dict:
        """Places a buy or sell order with the exchange."""
        return post(self, 'fapi/v1/order', {
            'symbol': market,
            'side': side,
            'type': order_type,
            'quantity': size,
            "reduceOnly": reduce_only
        })

In [3]:
class MarketStructure:
    """Represents the market structure of a crypto coin at one point in time."""
    
    def __init__(self, prev_high: tuple, prev_low: tuple, 
                 provisional_high: tuple, provisional_low: tuple,
                 recent_break: bool = False):
        self.msb = False
        self.stay_in_range = False
        self.continuation = False
        self.trend = False
        self.prev_high = prev_high
        self.prev_low = prev_low
        self.provisional_high = provisional_high
        self.provisional_low = provisional_low
        self.recent_break = recent_break
        
        
    def _break_up_trend(self, row):
        """Sets the new market structure after break of an up trend."""
        
        self.trend = False
        self.prev_high = self.provisional_high
        self.prev_low = tuple(row[['low', 'open time']])
        self.provisional_low = tuple(row[['low', 'open time']])
        logging.info('Up Trend Broken')
        
        
    def _break_down_trend(self, row):
        """Sets the new market structure after break of a down trend."""
        
        self.trend = True
        self.prev_high = tuple(row[['high', 'open time']])
        self.prev_low = self.provisional_low
        self.provisional_high = tuple(row[['high', 'open time']])
        logging.info('Down Trend Broken')
        
        
    def _continue_up_trend(self, row):
        """Sets the new market structure after an up trend continues."""
        
        self.prev_low = self.provisional_low
        self.prev_high = tuple(row[['high', 'open time']])
        self.provisional_high = tuple(row[['high', 'open time']])
        logging.info('Continuing Up Trend')
        
        
    def _continue_down_trend(self, row):
        """Sets the new market structure after a down trend continues."""
        
        self.prev_high = self.provisional_high
        self.prev_low = tuple(row[['low', 'open time']])
        self.provisional_low = tuple(row[['low', 'open time']])
        logging.info('Continuing Down Trend')
        
    def _stay_in_range(self, row):
        """Keeps track of provisional information while range bound."""
        
        if row['close'] - row['open'] >= 0:
            self.provisional_high = tuple(row[['high', 'open time']])
        if row['close'] - row['open'] < 0:
            self.provisional_low = tuple(row[['low', 'open time']])
        logging.info('Staying in Range')
        
        
    def next_candle(self, row):
        """Investigates how the market structure changes with the next incoming price data candle."""
        
        close = row['close']
        self.msb = False
        self.continuation = False
        self.stay_in_range = False
        trend = self.trend
        
        if close > self.prev_high[0]:
            if self.trend == False:
                self._break_down_trend(row)
                self.msb = True
            else:
                self._continue_up_trend(row)
                self.continuation = True
        elif close < self.prev_low[0]:
            if self.trend == False:
                self._continue_down_trend(row)
                self.continuation = True
            else:
                self._break_up_trend(row)
                self.msb = True
        else:
            self._stay_in_range(row)
            self.stay_in_range = True
            
        return trend, self.msb, self.continuation, self.stay_in_range

In [4]:
bc = BinanceClient(endpoint = 'https://fapi.binance.com/')
bc.open_session()
#bc.read_config()

In [5]:
no_of_sd = 2 #the number of SD's the amplitude of each component sine wave must be 
rsi_length = 14 #Rsi length
sample_size_power = 0 #2 to the power of this number will be the the amount of samples its broken up into

In [6]:
def fft(df: pd.DataFrame):
    #puts the data in a numpy array to be caculated
    data_array = np.array(df['rsi'])
    #sample size, usually a power of 2
    dt=2**sample_size_power
    # gets an array with the from 0 to the length of input data, every dt period
    t=np.arange(0,len(data_array),dt)

    #compute the FFT algo
    n = len(t)
    fhat = np.fft.fft(data_array, n)           #compute the FFT
    PSD = fhat * np.conj(fhat)/n               #power spectrum 
    freq = (1/(dt*n)) * np.arange(n)           #x-axis of frequencies 
    L = np.arange(1,np.floor(n/2),dtype='int') #only plot 1st half since the equation is symmeritrical 

    #checks if the array value of PSD is greater than 3 standard deviations and puts it into a true or false thing
    indices = PSD > np.std(PSD[L])*no_of_sd 
    #gets the intersects of the 3 sd array and the origional
    extract = np.extract(indices, PSD)

    #Use PSD to filter out noise
    PSDclean = PSD * indices   # zero out all others via kronecker delta
    fhat = indices*fhat        # zero out small fourier coffef. in Y
    ffilt = np.fft.ifft(fhat)  # inverse fFFT for filtered time signal

    #caculate gradient
    grad = np.gradient(ffilt)
    #puts the real ffilt numbers into a dataframe with the gradient and the corresponding time value
    signal = pd.DataFrame([ffilt.real,t,grad.real])
    #changes the axis so the columns are ffilt t and grad
    signal = signal.transpose()
    #sets the columns
    signal.columns = ['ffilt','t','grad']
    #gets rid of the t column
    signal = signal[['ffilt','grad']]
    return signal

In [15]:
class Trade:
    
    def __init__(self):
        self.prev_row = None
        self._equity = 100
        self.position_size = 0
        self.long_position = False
        self.short_position = False
        self.equity_curve = []
        self.equity_curve_no_zeros = []
        self.taker_fees = 0.0004

        
    @property
    def equity(self):
        """Returns the current equity value of the strategy."""
        
        if self.long_position or self.short_position:
            return (self.position_size*self.prev_row['close']) + self._equity
        else:
            return self._equity
        
        
    def long_trade(self, price):
        self.position_size = ((1-self.taker_fees)*self._equity)/price
        self._equity = 0
        self.long_position = True
        
    
    def short_trade(self, price):
        self.position_size = -((1-self.taker_fees)*self._equity)/price
        self._equity = self._equity - self.position_size*price
        self.short_position = True  
        
    
    def close_trade(self, price):                    
        self._equity = self._equity + price*self.position_size*(1.-self.taker_fees)
        self.position_size = 0
        self.long_position = False
        self.short_position = False

In [16]:
class RSITrade(Trade):
    """Implements the logical rules for a trend reversal trade triggered by an initial market structure break."""
             
    def __init__(self):
        super(RSITrade, self).__init__()

        
    def execute_trade(self, row, long: bool, short: bool):
        """Enters trade after triggered and follows through until trade exit."""
        
        #print(row)
        if self.long_position and short:
            self.close_trade(row['close'])
        if self.short_position and long:
            self.close_trade(row['close'])
        if long:
            self.long_trade(row['close'])
        if short:
            self.short_trade(row['close'])
            
        self.prev_row = row
        self.equity_curve_no_zeros.append(self.equity)

In [17]:
btc = {'market_name': 'BTCBUSD', 'ath': (58434.0, '2021-02-21 19:00:00+00:00'), 
      'prev_low': (57465.0, '2021-02-21 18:00:00+00:00'),
      'start': datetime(2021, 2, 21, 20, 0, 0, 0), 'initial_equity': 100, 'timeframe': '1h'}

market_name = btc['market_name']
timeframe = btc['timeframe']
start_time = btc['start']
end_time = start_time + timedelta(days=30)

In [18]:
def adjust_ts(ts):
    """Adjusts timestamps from milliseconds to seconds."""
    
    ts /= 1000
    return ts

In [19]:
    #response = bc.get(f'fapi/v1/klines?symbol={market_name}&interval={timeframe}'
    #                + f'&startTime={int(start_time.timestamp()*1000)}'
    #                + f'&endTime={int(end_time.timestamp()*1000)}&limit={1000}')
    #df = pd.DataFrame(response, columns = ['open time', 'open', 'high', 'low', 'close', 
    #                                        'volume', 'close time', 'quote asset volume', 
    #                                        'number of trades', 'taker buy base asset volume',
    #                                        'taker buy quote asset volume', 'unused field'])
        
    #df['open time'] = df['open time'].apply(adjust_ts)
    #df['open time'] = df['open time'].apply(datetime.fromtimestamp)
    #df[['open', 'high', 'low', 'close', 'volume']] \
    #    = df[['open', 'high', 'low', 'close', 'volume']].astype(float)   

In [23]:
import binance #pip install python-binance
trade = RSITrade()
client = binance.Client()
        
tmp = pd.DataFrame(client.get_historical_klines('BTCUSDT', client.KLINE_INTERVAL_1HOUR, 
                                                   '1-Jan-2020', '31-Oct-2022'), 
                      columns = ['open time', 'open', 'high', 'low', 'close', 
                                'volume', 'close time', 'quote asset volume', 
                                'number of trades', 'taker buy base asset volume',
                                'taker buy quote asset volume', 'unused field'])

In [ ]:
def backtest(tmp):     

    tmp['close'] = tmp['close'].astype(float)

    tmp['rsi'] = ta.momentum.RSIIndicator(tmp['close'], window=rsi_length).rsi()

    tmp = tmp.iloc[rsi_length-1:]
    tmp = tmp.reset_index().drop(columns='index')
    
    prev_sig = None

    for index, row in tmp[-8760:].iterrows():
 #   for index, row in tmp[-17520:].iterrows():

        #df = tmp[max(0,index-8760):index]
        
        df = tmp[:index]
        
        signal = fft(df)
        
        if prev_sig is None:
            prev_sig = signal['grad'].iloc[-2]
        sig = signal['grad'].iloc[-1]
        
        long = (sig > 0 and prev_sig < 0)
        short = (sig < 0 and prev_sig > 0)
        
        prev_sig = sig
        
        trade.execute_trade(df.iloc[-2], long, short)
        
backtest(tmp)

In [ ]:
plt.figure(figsize=(18,9))
plt.plot(trade.equity_curve_no_zeros)